# Installing Dependencies and Imports

In [1]:
!pip install nlp==0.4.0
!pip install transformers==3.5.1
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 212 kB 55.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 
     |████████████████████████████████| 1.1 MB 59.3 MB/s 
     |████████████████████████████████| 2.9 MB 49.0 MB/s 
     |████████████████████████████████| 880 kB 55.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=27c37117c8eaeaa33da7940c4c5790b3075a55a8c92ebbc1fffcfc4c33de8f06
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 6.9 kB/s 
 

In [2]:
from transformers import BertForSequenceClassification,BertTokenizerFast,Trainer,TrainingArguments
from nlp import load_dataset
import torch
import numpy as np

# Download and load the IMDB Dataset

In [3]:
!gdown https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc- 
dataset = load_dataset('csv',data_files='./imdbs.csv',split='train')

Downloading...
From: https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
To: /content/imdbs.csv
100% 132k/132k [00:00<00:00, 83.5MB/s]


Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-11046c2826f07a01/0.0.0/ede98314803c971fef04bcee45d660c62f3332e8a74491e0b876106f3d99bd9b. Subsequent calls will reuse this data.


In [4]:
type(dataset)

nlp.arrow_dataset.Dataset

In [5]:
dataset = dataset.train_test_split(test_size=0.3)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 70),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 30)}

In [7]:
train_set = dataset['train']
test_set = dataset['test']

# Initializing BERT for Sequence Classification

In [8]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Data Preprocessing

In [10]:
def preprocess(data):
 return tokenizer(data['text'], padding=True, truncation=True)

In [11]:
train_set = train_set.map(preprocess, batched=True,batch_size=len(train_set))
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
train_set.set_format('torch',columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch',columns=['input_ids', 'attention_mask', 'label'])

# Initializing Hyperparameters and Trainer arguments

In [18]:
batch_size = 8
epochs = 20
warmup_steps = 500
weight_decay = 0.01

In [19]:
training_args = TrainingArguments(
 output_dir='./results',
 num_train_epochs=epochs,
 per_device_train_batch_size=batch_size,
 per_device_eval_batch_size=batch_size,
 warmup_steps=warmup_steps,
 weight_decay=weight_decay,
 evaluate_during_training=True,
 logging_dir='./logs',
)

In [20]:
trainer = Trainer(
 model=model,
 args=training_args,
 train_dataset=train_set,
 eval_dataset=test_set
)

# Training and Evaluation

In [21]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=180, training_loss=0.37738172743055554)

In [22]:
trainer.evaluate()

{'eval_loss': 0.6592559814453125, 'epoch': 20.0}